In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
import DataSet
import Evaluation
import datetime
import numpy as np 
import matplotlib.pyplot as plt
import random
from matplotlib.backends.backend_pdf import PdfPages

from Utils import *

from DataSet import UniHHIMUGestures
from torch.utils.data import Dataset, DataLoader

We need to specify a bunch of parameters the expriment:

In [ ]:
#===========================================================================
# Give this run a name. 
# If name equals 'test', no log will be generated
#===========================================================================
name = 'test'


#===========================================================================
# Decide which gesture data shall be used for training
#===========================================================================
inputGestures = [0,1,2,3,4,5,6,7,8,9]

#===========================================================================
# Decide which target signals shall be used for training
#===========================================================================
usedGestures = [0,1,2,3,4,5,6,7,8,9]

#===========================================================================
# Concatenate data to create "more" training samples, 1 corresponds to no concatenations
#===========================================================================
concFactor = 1

#===========================================================================
# Add noise to the data, 0 corresponds to no noise. Noise above 2 has shown to weaken recognition
#===========================================================================
noiseFactor = 1

#===========================================================================
# Decide wether gestures shall be shuffled before training. If true, nFolds many 
# pieces will be generated. Not every piece is garanteed to contain every gesture, so do not use too many.
#===========================================================================
shuffle = True
nFolds = 4


#===========================================================================
# Function used to evaluate during cross validation. Possible functions are:
# Evaluation.calc1MinusF1FromMaxApp (best working, used in thesis)
# Oger.utils.nmse (normalised mean square error, tells nothing about classifier perfomance but works okay)
# Evaluation.calcLevenshteinError (use the Levenshtein error, disadvantages are highlighted in thesis) 
# Evaluation.calc1MinusF1FromInputSegment (use segmentation by supervised signal)
#===========================================================================
evaluationFunction = Evaluation.calc1MinusF1FromMaxApp

#===========================================================================
# Set this to true if another output neuron shall be added to represent "no gesture"
#===========================================================================
learnTreshold = True

#===========================================================================
# Use on of the optimisation dictionaries from the optDicts file
#===========================================================================
optDict = 'bestParas'

#===========================================================================
# Use normalizer
#===========================================================================
useNormalized = 2

#===========================================================================
# Pick datasets to train on, and datasets to test on
#===========================================================================
inputFiles = ['nike','julian','nadja','line']
testFiles = ['stephan']

# If desired add a specific file to test on, e.g. randTestFiles = ['lana_0_0.npz']
randTestFiles = []



#===========================================================================
# Setup project directory
#===========================================================================
now = datetime.datetime.now()
resultsPath = getProjectPath()+'results/'
pdfFileName = now.strftime("%Y-%m-%d-%H-%M")+'_'+name+'.pdf'
pdfFilePath = resultsPath+'pdf/'+pdfFileName
npzFileName = now.strftime("%Y-%m-%d-%H-%M")+'_'+name+'.npz'
npzFilePath = resultsPath+'npz/'+npzFileName
bestFlowPath = resultsPath+'nodes/'+now.strftime("%Y-%m-%d-%H-%M")+'_'+name+'.p'
pp = PdfPages(pdfFilePath)


#===========================================================================
# Add labels for gestures
#===========================================================================
totalGestureNames = ['left','right','forward','backward','bounce up','bounce down','turn left','turn right','shake lr','shake ud', \
                     'tap 1','tap 2','tap 3','tap 4','tap 5','tap 6','no gesture']
gestureNames = []
for i in usedGestures:
    gestureNames.append(totalGestureNames[i])
gestureNames.append('no gesture')

Create dataset and dataloaders

In [ ]:
trainset = UniHHIMUGestures(dataDir='dataSets/', train=True, useNormalized=useNormalized)
testset = UniHHIMUGestures(dataDir='dataSets/', train=False, useNormalized=useNormalized)

trainloader = DataLoader(trainset, batch_size=1,
                        shuffle=True, num_workers=1)
testloader = DataLoader(testset, batch_size=1,
                        shuffle=True, num_workers=1)

Let's take a look at the scaled input data:

In [ ]:
fig, ax = plt.subplots(3,1, figsize=(20,9))
ax[0].plot(trainset[0][0][:800,0:3])
ax[1].plot(trainset[0][0][:800,3:6])
ax[2].plot(trainset[0][0][:800,6:9])
plt.tight_layout()

Looks all good, we can clearly see the gesture sequences intercepted by non gesture seqeuences in between.
Now let's create a Leaky Integrator ESN with parameters as defined in the paper.

In [ ]:
from echotorch.nn.ESN import ESN
from echotorch.nn.LiESN import LiESN



esn = LiESN(
    input_dim=9,
    hidden_dim=400,
    output_dim=10,
    input_scaling=13.,
    sparsity=0.1, # input matrix sparsity
    w_distrib='gaussian',
    spectral_radius=1.,
    bias_scaling=0., # no input bias
    feedbacks=False, # No feedback connections
    learning_algo='inv',
    leaky_rate=.3
)



## Training

Training is done automatically when presenting the values and targets to the network. After calling finalize training is completed and network switches to prediction mode.

In [ ]:
from torch.autograd import Variable

for inputs, targets in trainloader:
    inputs, targets = Variable(inputs.float()), Variable(targets.float())
    esn(inputs, targets)
    
esn.finalize()

## Testing

Plot activations of network on testset and corresponding target signals.

In [ ]:
for test_inputs, test_targets in testloader:
    plt.figure(figsize=(20,5))
    outputs = esn(test_inputs.float())
    plt.plot(outputs[0,:800,:])
    plt.plot(test_targets[0,:800,:])
plt.tight_layout()

In [ ]:
import sklearn
trainCms = []
trainF1MaxApps = []
trainPredicitions = []
trainTargets = []
trainF1s = []

prediction = outputs[0].numpy()
t_target =  test_targets[0].numpy()

pred, targ = Evaluation.calcInputSegmentSeries(prediction, t_target, 0.4, False)
trainF1s.append(np.mean(sklearn.metrics.f1_score(targ,pred,average=None)))


t_maxApp_prediction = Evaluation.calcMaxActivityPrediction(prediction,t_target,0.5,10)
pred_MaxApp, targ_MaxApp = Evaluation.calcInputSegmentSeries(t_maxApp_prediction, t_target, 0.5)
trainF1MaxApps.append(np.mean(sklearn.metrics.f1_score(targ_MaxApp,pred_MaxApp,average=None)))

conf = sklearn.metrics.confusion_matrix(targ_MaxApp, pred_MaxApp)
trainCms.append(conf)
trainPredicitions.append(prediction)
trainTargets.append(t_target)

Evaluation.plot_confusion_matrix(trainCms[0], gestureNames, 'test set')
plt.tight_layout()
plt.ylim(10.5,-0.5)

print("Test f1 score for maxactivity: {:.2f}".format(trainF1MaxApps[0]))

#### TODO: why is the testscore here lower than in OGER? supposed to be 0.8

# LSTM

Create and train LSTM on the give data.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
class LSTMClassifier(nn.Module):

    def __init__(self, hidden_dim=25):
        super(LSTMClassifier, self).__init__()
        
        self.lstm = torch.nn.LSTM(input_size=9, hidden_size=hidden_dim, batch_first=True)
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim,10, bias=False),
            #torch.nn.Sigmoid()
        )


    def forward(self, inputs):
        
        
        lstm_outputs, (hidden_acts, cell_states) = self.lstm(inputs)
        predictions = self.classifier(lstm_outputs)
        return predictions

In [ ]:
lstm = LSTMClassifier(25)
loss_function = torch.nn.MSELoss()
optimizer = optim.SGD(lstm.parameters(), lr=1e-1)


for epoch in range(10):
    for inputs, targets in trainloader:

        inputs, targets = inputs.float(), targets.float()
        
        inputs[0,:,:9] = targets[0,:,:9]

        lstm.zero_grad()

        outputs =  lstm(inputs)


        loss = loss_function(outputs, targets)
        loss.backward()
        
        lstm.lstm
        
        optimizer.step()
    if epoch % 2 == 0:
        print(loss.item())
        lstm.eval()
        plt.figure(figsize=(20,5))
        plt.title('Epoch: {:}, loss: {:.4f}'.format(epoch, loss))
        test_preds = lstm(test_inputs.float()).detach().numpy()[0]
        plt.plot(test_preds[:800,:])
        plt.plot(test_targets[0,:800,:])
        plt.ylim(-0.1,1.1)
        plt.pause(1)
        lstm.train(True)

In [ ]:
lstm.eval()
plt.figure(figsize=(20,5))
test_preds = lstm(test_inputs.float()).detach().numpy()[0]
plt.plot(test_preds[:800,:])
plt.pause(1)
